In [109]:
import pandas as pd
from pandas import isnull
import numpy as np
from os import path

In [110]:
def mem_mib(df):
    print("{0:.2f} Mib".format(df.memory_usage().sum()/(1024*1024)))

def make_categorical(df, col_name):
    df.loc[:, col_name] = pd.Categorical(df[col_name])
    
def recent_nhl_only(df):
    return df[(df["lgID"] == "NHL") & (df["year"] >= 1980)]

In [111]:
file_name = path.join(path.curdir,'../data/external/advpandas/Master.csv')

In [112]:
master = pd.read_csv(file_name)
master.head()

,playerID,coachID,hofID,firstName,lastName,nameNote,nameGiven,nameNick,height,weight,...,birthDay,birthCountry,birthState,birthCity,deathYear,deathMon,deathDay,deathCountry,deathState,deathCity
0,aaltoan01,NaN,NaN,Antti,Aalto,NaN,Antti,NaN,73.0,210.0,...,4.0,Finland,NaN,Lappeenranta,NaN,NaN,NaN,NaN,NaN,NaN
1,abbeybr01,NaN,NaN,Bruce,Abbey,NaN,Bruce,NaN,73.0,185.0,...,18.0,Canada,ON,Toronto,NaN,NaN,NaN,NaN,NaN,NaN
2,abbotge01,NaN,NaN,George,Abbott,NaN,George Henry,Preacher,67.0,153.0,...,3.0,Canada,ON,Synenham,NaN,NaN,NaN,NaN,NaN,NaN
3,abbotre01,NaN,NaN,Reg,Abbott,NaN,Reginald Stewart,NaN,71.0,164.0,...,4.0,Canada,MB,Winnipeg,NaN,NaN,NaN,NaN,NaN,NaN
4,abdelju01,NaN,NaN,Justin,Abdelkader,NaN,NaN,NaN,73.0,195.0,...,25.0,USA,MI,Muskegon,NaN,NaN,NaN,NaN,NaN,NaN


In [113]:
#Check for null values in playerId Column
master['playerID'].pipe(isnull).value_counts()
master_orig = master.copy()
# Drop the records where playerID is null
master = master.dropna(subset=["playerID"])
master = master.dropna(subset=["lastNHL","firstNHL"], how="all")
# Keep only last 40 years of data for NHL
master = master.loc[master["lastNHL"]>=1980]
master.shape

(4627, 31)

In [114]:
# Remove the unwanted columns
master = master.filter(regex="(playerID|pos|^birth)|(Name$)")

In [115]:
mem_mib(master)
mem_mib(master_orig)

0.39 Mib
1.84 Mib


In [116]:
make_categorical(master, "pos")
make_categorical(master, "birthCountry")
make_categorical(master, "birthState")
master.set_index("playerID")
mem_mib(master)
p_file_name = path.join(path.curdir,'../data/external/advpandas/processed/Master.pickle')
master.to_pickle(p_file_name)
master.head()

0.30 Mib


,playerID,firstName,lastName,pos,birthYear,birthMon,birthDay,birthCountry,birthState,birthCity
0,aaltoan01,Antti,Aalto,C,1975.0,3.0,4.0,Finland,NaN,Lappeenranta
4,abdelju01,Justin,Abdelkader,L,1987.0,2.0,25.0,USA,MI,Muskegon
9,abidra01,Ramzi,Abid,L,1980.0,3.0,24.0,Canada,QC,Montreal
11,abrahth01,Thommy,Abrahamsson,D,1947.0,4.0,12.0,Sweden,NaN,Leksand
14,actonke01,Keith,Acton,C,1958.0,4.0,15.0,Canada,ON,Stouffville
